### Create DB in pgAdmin:
* in your pgAdmin Create database `spotify_db` 
* Execute this sql to create the three tables we will Extract and Transform

```sql
CREATE TABLE spotify_2017 (
    artists TEXT PRIMARY KEY,
    name_count_2017 INT,
    danceability_2017 FLOAT,
    energy_2017 FLOAT
);


CREATE TABLE spotify_2018 (
    artists TEXT PRIMARY KEY,
    name_count_2018 INT,
    danceability_2018 FLOAT,
    energy_2018 FLOAT
);

CREATE TABLE spotify_2017_2018 (
    artists TEXT PRIMARY KEY,
    name_count_2017 INT,
    name_count_2018 INT,
    YoY_danceability FLOAT,
    YoY_energy FLOAT,
);

```

In [11]:
import pandas as pd
from sqlalchemy import create_engine

## Extract

In [12]:
#Extract each csv (one for 2017 and 2018)and preview data with .head()
top_songs_2017 = "top2017.csv"
df_2017 = pd.read_csv(top_songs_2017)
df_2017.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,7qiZfU4dY1lWllzX7mPBI,Shape of You,Ed Sheeran,0.825,0.652,1.0,-3.183,0.0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,233713.0,4.0
1,5CtI0qwDJkDQGwXD1H1cL,Despacito - Remix,Luis Fonsi,0.694,0.815,2.0,-4.328,1.0,0.1200,0.2290,0.000000,0.0924,0.813,88.931,228827.0,4.0
2,4aWmUDTfIPGksMNLV2rQP,Despacito (Featuring Daddy Yankee),Luis Fonsi,0.660,0.786,2.0,-4.757,1.0,0.1700,0.2090,0.000000,0.1120,0.846,177.833,228200.0,4.0
3,6RUKPb4LETWmmr3iAEQkt,Something Just Like This,The Chainsmokers,0.617,0.635,11.0,-6.769,0.0,0.0317,0.0498,0.000014,0.1640,0.446,103.019,247160.0,4.0
4,3DXncPQOG4VBw3QHh3S81,I'm the One,DJ Khaled,0.609,0.668,7.0,-4.284,1.0,0.0367,0.0552,0.000000,0.1670,0.811,80.924,288600.0,4.0


In [13]:
#Extract each csv (one for 2017 and 2018) and preview data with .head()
top_songs_2018 = "top2018.csv"
df_2018 = pd.read_csv(top_songs_2018)
df_2018.head()

,id,name,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6DCZcSspjsKoFjzjrWoCd,God's Plan,Drake,0.754,0.449,7.0,-9.211,1.0,0.1090,0.0332,0.000083,0.552,0.357,77.169,198973.0,4.0
1,3ee8Jmje8o58CHK66QrVC,SAD!,XXXTENTACION,0.740,0.613,8.0,-4.880,1.0,0.1450,0.2580,0.003720,0.123,0.473,75.023,166606.0,4.0
2,0e7ipj03S05BNilyu5bRz,rockstar (feat. 21 Savage),Post Malone,0.587,0.535,5.0,-6.090,0.0,0.0898,0.1170,0.000066,0.131,0.140,159.847,218147.0,4.0
3,3swc6WTsr7rl9DqQKQA55,Psycho (feat. Ty Dolla $ign),Post Malone,0.739,0.559,8.0,-8.011,1.0,0.1170,0.5800,0.000000,0.112,0.439,140.124,221440.0,4.0
4,2G7V7zsVDxg1yRsu7Ew9R,In My Feelings,Drake,0.835,0.626,1.0,-5.833,1.0,0.1250,0.0589,0.000060,0.396,0.350,91.030,217925.0,4.0


### Transform
Join to get a comparison table between 2017 and 2018

In [14]:
# Just pull in necessary columns
df_2017 = df_2017[['id','name','artists','danceability','energy']]

#Round danceability to the nearest 100th and accoustic to the nearest 1000th
df_2017['danceability'] = df_2017['danceability'].round(2)
df_2017['energy'] = df_2017['energy'].round(3)

#Group by with an aggregate function per column
df_2017_agg = df_2017.groupby('artists').agg({'name':'count', 
                         'danceability':'mean', 
                         'energy':'mean'})

# Rename the column headers
df_2017_agg.rename(columns={"name": "name_count_2017",\
                            "danceability": "danceability_2017",\
                            "energy": "energy_2017"}, inplace = True)

# Sort by name so the artists with the most songs is at the top
df_2017_agg.sort_values(['name_count_2017'], ascending=False, inplace=True)

# Set index
df_2017_agg.reset_index(inplace=True)

#Preview data
df_2017_agg

,artists,name_count_2017,danceability_2017,energy_2017
0,Ed Sheeran,4,0.625000,0.702500
1,The Chainsmokers,4,0.640000,0.669000
2,Drake,3,0.843333,0.523333
3,Martin Garrix,3,0.560000,0.556333
4,Maroon 5,2,0.740000,0.666500
...,...,...,...,...
73,Julia Michaels,1,0.710000,0.427000
74,Justin Bieber,1,0.740000,0.739000
75,KYLE,1,0.750000,0.653000
76,AJR,1,0.670000,0.637000


In [15]:
# Just pull in necessary columns
df_2018 = df_2018[['id','name','artists','danceability','energy']]

#Round danceability to the nearest 100th and accoustic to the nearest 1000th
df_2018['danceability'] = df_2018['danceability'].round(2)
df_2018['energy'] = df_2018['energy'].round(3)

#Group by with an aggregate function per column
df_2018_agg = df_2018.groupby('artists').agg({'name':'count', 
                         'danceability':'mean', 
                         'energy':'mean'})

# Rename the column headers
df_2018_agg.rename(columns={"name": "name_count_2018",\
                            "danceability": "danceability_2018",\
                            "energy": "energy_2018"}, inplace = True)

# Sort by name so the artists with the most songs is at the top
df_2018_agg.sort_values(['name_count_2018'], ascending=False, inplace=True)

# Set index
df_2018_agg.reset_index(inplace=True)

#Preview data
df_2018_agg

,artists,name_count_2018,danceability_2018,energy_2018
0,Post Malone,6,0.645000,0.608833
1,XXXTENTACION,6,0.788333,0.492000
2,Drake,4,0.772500,0.599000
3,Ed Sheeran,3,0.670000,0.466333
4,Marshmello,3,0.613333,0.811000
...,...,...,...,...
65,Lauv,1,0.750000,0.505000
66,6ix9ine,1,0.930000,0.387000
67,Lil Dicky,1,0.760000,0.599000
68,Lil Uzi Vert,1,0.730000,0.750000


In [16]:
df_agg = pd.merge(df_2017_agg,df_2018_agg)

In [17]:
#insert formulas to compare YoY
df_agg['YoY_song_growth'] = df_agg['name_count_2018'] - df_agg['name_count_2017']
df_agg['YoY_danceability'] = df_agg['danceability_2018'] - df_agg['danceability_2017']
df_agg['YoY_energy'] = df_agg['energy_2018'] - df_agg['energy_2017']

In [18]:
#round to make more legible
df_agg['YoY_danceability'] = df_agg['YoY_danceability'].round(5)
df_agg['YoY_energy'] = df_agg['YoY_energy'].round(5)

In [19]:
#create new comparison table
df_agg_summary = df_agg[['artists', 'name_count_2018', 'name_count_2017', 'YoY_song_growth', 'YoY_danceability', 'YoY_energy']]

In [29]:
df_agg_summary

,artists,name_count_2018,name_count_2017,YoY_song_growth,YoY_danceability,YoY_energy
0,Ed Sheeran,3,4,-1,0.04500,-0.23617
1,Drake,4,3,1,-0.07083,0.07567
2,Maroon 5,2,2,0,0.08500,-0.09750
3,DJ Khaled,1,2,-1,-0.09000,0.09000
4,Clean Bandit,2,2,0,-0.02500,-0.04900
5,Imagine Dragons,2,2,0,0.00000,0.00250
6,Post Malone,6,2,4,0.04000,-0.05817
7,Calvin Harris,2,2,0,-0.03000,0.04500
8,Luis Fonsi,1,2,-1,0.05500,0.08850
9,Bruno Mars,1,2,-1,-0.13500,0.17750


### Create database connection

In [21]:
connection_string = "{username}:{password}@localhost:5432/spotify_db"
engine = create_engine(f'postgresql://{connection_string}')
#Confirm tables
engine.table_names()

ModuleNotFoundError: No module named 'psycopg2'

### Load DataFrames into database

In [22]:
df_2017_agg.to_sql(name='spotify_2017', con=engine, if_exists='replace', index=True)
df_2018_agg.to_sql(name='spotify_2018', con=engine, if_exists='replace', index=True)
df_agg_summary.to_sql(name='spotify_2017_2018', con=engine, if_exists='replace', index=True)

NameError: name 'engine' is not defined

### SQL Query Examples


```sql
---1 Essential create the join table that was created in python ("df_agg"), but in sql.
SELECT spotify_2017.artists, spotify_2017.name_count_2017, spotify_2017.danceability_2017, spotify_2017.energy_2017,spotify_2018.name_count_2018, spotify_2018.danceability_2018, spotify_2018.energy_2018 
FROM spotify_2017 JOIN spotify_2018 ON spotify_2017.artists = spotify_2018.artists;


---2 List artists who had more songs in 2018 vs. 2017 and compare if their is a change in their danceability or energy YoY.
SELECT artists, name_count_2018, name_count_2017, YoY_song_growth, YoY_danceability, YoY_energy
FROM spotify_2017_2018 WHERE YoY_song_growth BETWEEN 1 AND 5;

---3 List artists whose danceability decreased in 2018 vs. 2017.
SELECT artists, YoY_song_growth, YoY_danceability
FROM spotify_2017_2018 WHERE YoY_danceability BETWEEN -1 AND 0;

```